<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/02-Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [ ]:
!pip install -q openai==1.12.0 cohere==4.47 tiktoken==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [ ]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False

# Load Dataset

## Download Dataset (JSON)

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.

In [ ]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

--2024-03-20 16:18:39--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.02s   

2024-03-20 16:18:40 (6.91 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]

--2024-03-20 16:18:40--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP r

## Read File

In [ ]:
# Split the input text into chunks of specified size.
def split_into_chunks(text, chunk_size=1024):
  chunks = []
  for i in range(0, len(text), chunk_size):
    chunks.append( text[i:i+chunk_size] )

  return chunks

In [ ]:
import csv

chunks = []

# Load the file as a CSV
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    chunks.extend( split_into_chunks(row[1]) )

In [ ]:
import pandas as pd

# Convert the JSON list to a Pandas Dataframe
df = pd.DataFrame(chunks, columns=['chunk'])

df.keys()

Index(['chunk'], dtype='object')

# Generate Embedding

In [ ]:
from openai import OpenAI

client = OpenAI()

# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
  try:
    # Remove newlines
    text = text.replace("\n", " ")
    res = client.embeddings.create(input = [text], model="text-embedding-ada-002")

    return res.data[0].embedding

  except:
        return None

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

# Generate embedding
if not load_embedding:
  print("Generating embeddings...")
  embeddings = []
  for index, row in tqdm( df.iterrows() ):
    # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
    embeddings.append( get_embedding( row['chunk'] ) )

  embeddings_values = pd.Series(embeddings)
  df.insert(loc=1, column='embedding', value=embeddings_values)

# Or, load the embedding from the file.
else:
  print("Loaded the embedding file.")
  # Load the file as a CSV
  df = pd.read_csv('mini-llama-articles-with_embeddings.csv')
  # Convert embedding column to an array
  df['embedding'] = df['embedding'].apply(lambda x: np.array(eval(x)), 0)

Generating embeddings...


0it [00:00, ?it/s]

In [ ]:
# df.to_csv('mini-llama-articles-with_embeddings.csv')

# User Question

In [ ]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding( QUESTION )

len( QUESTION_emb )

1536

# Test Cosine Similarity

Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.

In [ ]:
BAD_SOURCE_emb = get_embedding( "The sky is blue." )
GOOD_SOURCE_emb = get_embedding( "LLaMA2 model has a total of 2B parameters." )

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]) )
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]) )

> Bad Response Score: [[0.69953438]]
> Good Response Score: [[0.93126147]]


# Calculate Cosine Similarities

In [ ]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity( [QUESTION_emb], df['embedding'].tolist() )

print( cosine_similarities )

[[0.82047387 0.79858187 0.74135248 0.73226232 0.72406104 0.75608299
  0.76808965 0.77621683 0.80498431 0.71399955 0.69822549 0.67532971
  0.72473021 0.73449361 0.69998132 0.73749561 0.68490681 0.75076836
  0.72540663 0.70675593 0.76047822 0.73849418 0.78103858 0.75189435
  0.73619013 0.76962672 0.71289635 0.76996122 0.7827543  0.77959332
  0.82716952 0.77719335 0.80172766 0.76301732 0.78111546 0.75179235
  0.74741505 0.7576328  0.78998865 0.77283347 0.79180172 0.78170323
  0.80264132 0.79923073 0.76146584 0.75199024 0.8341403  0.74460259
  0.76259332 0.73693499 0.78469623 0.81698455 0.8254561  0.77921093
  0.75351863 0.79319721 0.73098248 0.71716001 0.73210099 0.74684248
  0.75760574 0.71070101 0.71507394 0.70847896 0.72395535 0.77801292
  0.75446732 0.75100258 0.7361131  0.78430831 0.74170516 0.71862961
  0.76792911 0.76471996 0.78551313 0.80846857 0.79231644 0.79505895
  0.76910825 0.78341548 0.74952152 0.7849115  0.80407507 0.82641741
  0.77074756 0.7356681  0.77452715 0.76224969 0.

In [ ]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax( cosine_similarities )

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print( indices )

[114  89  91]


In [ ]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate( df.chunk[indices] ):
  print(f"> Chunk {idx+1}")
  print(item)
  print("----")

> Chunk 1
by Meta that ventures into both the AI and academic spaces. The model aims to help researchers, scientists, and engineers advance their work in exploring AI applications. It will be released under a non-commercial license to prevent misuse, and access will be granted to academic researchers, individuals, and organizations affiliated with the government, civil society, academia, and industry research facilities on a selective case-by-case basis. The sharing of codes and weights allows other researchers to test new approaches in LLMs. The LLaMA models have a range of 7 billion to 65 billion parameters. LLaMA-65B can be compared to DeepMind's Chinchilla and Google's PaLM. Publicly available unlabeled data was used to train these models, and training smaller foundational models require less computing power and resources. LLaMA 65B and 33B have been trained on 1.4 trillion tokens in 20 different languages, and according to the Facebook Artificial Intelligence Research (FAIR) team,

# Augment the Prompt

In [ ]:
# Use the OpenAI API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
      "You are an assistant and expert in answering questions from a chunks of content. "
      "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
      "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
      "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format( "".join( df.chunk[indices] ), QUESTION )

    # Call the OpenAI API
    response = client.chat.completions.create(
      model='gpt-3.5-turbo-16k',
      temperature=0.0,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
      ]
    )

    # Return the AI's response
    res = response.choices[0].message.content.strip()

except Exception as e:
    print( f"An error occurred: {e}" )

In [ ]:
print( res )

The LLaMA2 model has four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.


# Without Augmentation

Test the OpenAI API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.

In [ ]:
# Formulating the system prompt
system_prompt = (
  "You are an assistant and expert in answering questions."
)

# Combining the system prompt with the user's question
prompt = (
  "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
)
prompt = prompt.format( QUESTION )

# Call the OpenAI API
response = client.chat.completions.create(
  model='gpt-3.5-turbo-16k',
  temperature=.9,
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
  ]
)

# Return the AI's response
res = response.choices[0].message.content.strip()

In [ ]:
print( res )

The LLaMA2 model has a total of [insert number] parameters.
